In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn import linear_model


import pandas as pd
import numpy as np

## Reading Data

In [ ]:
df_test=pd.read_csv('../input/test.csv')
df_train=pd.read_csv('../input/train.csv')
df_history = pd.read_csv('../input/historical_transactions.csv')
df_nmerchant = pd.read_csv('../input/new_merchant_transactions.csv')

In [ ]:
valid_card_id = pd.concat([df_train.card_id, df_test.card_id])
df_history = df_history[df_history.card_id.isin(valid_card_id)]
df_nmerchant = df_nmerchant[df_nmerchant.card_id.isin(valid_card_id)]
df_history.shape, df_nmerchant.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(valid_card_id)
df_train.card_id = le.transform(df_train.card_id)
df_test.card_id = le.transform(df_test.card_id)
df_history.card_id = le.transform(df_history.card_id)
df_nmerchant.card_id = le.transform(df_nmerchant.card_id)

In [ ]:
def delete_day(date):
    x = date.split('-')[0] + '-' + date.split('-')[1] 
    return x

In [ ]:
# 1998-10-01
def get_year(date):
    x = date.split('-')[0] 
    return x

In [ ]:
#10:06:26 - 10
def get_hour(time):
    x = time.split(':')[0] 
    return x        

In [ ]:
#2018-03-11 14:57:36
def get_time(datetime):
    time = datetime.split(' ')[1] 
    return time

In [ ]:
def get_date(datetime):
    date = datetime.split(' ')[0] 
    return date

In [ ]:
def AGG_DIFF(x):
    res = 0
    prev_elem = x.iloc[0]
    for elem in x:
        res += elem - prev_elem
        prev_elem = elem
    return res

In [ ]:
#1. надо, по крайней мере, что-то сделать со столбцами, в которых записаны строковые значения.
#2. подозрения всегда вызывают столбцы с небольшим числом уникальных значений
# найти все признаки, в которых первое значение - строка
def find_cat(df):
    for col_name in df.columns:
        s = ''
        s += name
        if (type(data[name][0]) == str):
            s += ' строка,'
        if (data[name].nunique()<=5):
            s += ' мало уникальных'
        if (s!=name):
            print (s)

In [ ]:
def preproc_for_cat_data(df):
    dct = {'N': 0, 'Y': 1}
    df['category_1'] = df['category_1'].map(dct) 
    #dct = {'A': 3, 'B': 2, 'C': 1}
    dct = {i:len(df[df.category_3==i])/len(df) for i in df.category_3.unique()}
    df['category_3'] = df['category_3'].map(dct)

    df['TIME'] = df.purchase_date.map(lambda x: get_time(x))
    df['DATE'] = df.purchase_date.map(lambda x: get_date(x))
    df['category_2']=df['category_2'].fillna(0) 
    df.drop(['purchase_date'], axis=1, inplace=True)
    df['year'] = df.DATE.map(lambda x: get_year(x))
    df['YM'] = df.DATE.map(lambda x: delete_day(x))
    return df

In [ ]:
df_history = preproc_for_cat_data(df_history)

In [ ]:
import gc

In [ ]:
#gc.collect()

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
df_history = reduce_mem_usage(df_history)

In [ ]:
for n in ('purchase_amount', 'category_1', 'installments', 'merchant_category_id'):
    print (n)
    df_train=df_train.merge(df_history.pivot_table
                        (values=n, index=["card_id"], columns=["YM"], 
                         aggfunc=np.sum, dropna=True), 'left', on='card_id')
    #('merchant_category_id', '2017-02') - {'2017-02': 'merchant_category_id_2017-02'}
    dct = {i[1]:i[0]+'_'+i[1] for i in df_history.pivot_table(values=[n], index=["card_id"], columns=["YM"]
                       , aggfunc=np.sum, dropna=True).columns}
    df_train.rename(columns=dct, inplace=True)
    #
    df_test=df_test.merge(df_history.pivot_table
                        (values=n, index=["card_id"], columns=["YM"], 
                         aggfunc=np.sum, dropna=True), 'left', on='card_id')
    df_test.rename(columns=dct, inplace=True)
#
#  
df_train=df_train.merge(df_history.groupby('card_id')
                        ['category_2', 'category_3'].agg([np.sum, np.mean, np.max]), 'left', on='card_id')

df_test=df_test.merge(df_history.groupby('card_id')
                        ['category_2', 'category_3'].agg([np.sum, np.mean, np.max]), 'left', on='card_id')
    
df_train.shape, df_test.shape

In [ ]:
del df_history

In [ ]:
#gc.collect()

In [ ]:
df_merchant=pd.read_csv('../input/merchants.csv')
dct = {'N': 0, 'Y': 1}
df_merchant['category_1'] = df_merchant['category_1'].map(dct)
df_merchant['category_4'] = df_merchant['category_4'].map(dct)

#dct = {'A': 5, 'B': 4, 'C': 3, 'D': 2, 'E': 1}
dct = {i:len(df_merchant[df_merchant.most_recent_sales_range==i])
       /len(df_merchant) for i in df_merchant.most_recent_sales_range.unique()}
df_merchant['most_recent_sales_range'] = df_merchant['most_recent_sales_range'].map(dct)
dct = {i:len(df_merchant[df_merchant.most_recent_purchases_range==i])
       /len(df_merchant) for i in df_merchant.most_recent_purchases_range.unique()}
df_merchant['most_recent_purchases_range'] = df_merchant['most_recent_purchases_range'].map(dct)
mean = round(df_merchant['avg_sales_lag3'].mean(), 5)
df_merchant['avg_sales_lag3']=df_merchant['avg_sales_lag3'].fillna(mean)
mean = round(df_merchant['avg_sales_lag6'].mean(), 5)
df_merchant['avg_sales_lag6']=df_merchant['avg_sales_lag6'].fillna(mean)
mean = round(df_merchant['avg_sales_lag12'].mean(), 5)
df_merchant['avg_sales_lag12']=df_merchant['avg_sales_lag12'].fillna(mean)

df_merchant.head()

In [ ]:
#df_nmerchant = pd.read_csv('../input/new_merchant_transactions.csv')
df_nmerchant = preproc_for_cat_data(df_nmerchant)
df_merchant.shape, df_nmerchant.shape

In [ ]:
#df_nm = 
df_nmerchant = df_nmerchant.merge(df_merchant, how='left', on=['merchant_id', 'city_id', 'state_id']) #, 'city_id', 'state_id'
#inner - (1968679, 38) используйте пересечение ключей из обоих фреймов, сохраняйте порядок левых ключей
#(1436087, 36) on=['merchant_id', 'city_id', 'state_id']
#left -  (1994895, 38) используйте только ключи из левого фрейма,
#(1970812, 36)
df_nmerchant.shape

In [ ]:
for n in ('purchase_amount', 'category_1_y', 'installments', 'merchant_category_id_y', 'category_3'):
    print (n)
    df_train=df_train.merge(df_nmerchant.pivot_table
                        (values=n, index=["card_id"], columns=["YM"], 
                         aggfunc=np.sum, dropna=True), 'left', on='card_id')

    dct = {i[1]:i[0]+'_'+i[1] for i in df_nmerchant.pivot_table(values=[n], index=["card_id"], columns=["YM"]
                       , aggfunc=np.sum, dropna=True).columns}
    df_train.rename(columns=dct, inplace=True)
    #
    df_test=df_test.merge(df_nmerchant.pivot_table
                        (values=n, index=["card_id"], columns=["YM"], 
                         aggfunc=np.sum, dropna=True), 'left', on='card_id')
    df_test.rename(columns=dct, inplace=True)

df_train=df_train.merge(df_nmerchant.groupby('card_id')
                        ['category_2_y'].agg([np.sum, np.mean]), 'left', on='card_id')

df_test=df_test.merge(df_nmerchant.groupby('card_id')
                        ['category_2_y'].agg([np.sum, np.mean]), 'left', on='card_id')
    
df_train.shape, df_test.shape

In [ ]:
df_nmerchant.head()

In [ ]:
df_train=df_train.merge(df_nmerchant.groupby('card_id')
                        ['merchant_category_id_y'
                         , 'numerical_1',  'numerical_2', 'avg_sales_lag3', 'avg_purchases_lag3'
                         , 'active_months_lag3', 'avg_sales_lag6', 'avg_purchases_lag6', 'active_months_lag6'
                         , 'avg_sales_lag12', 'avg_purchases_lag12', 'active_months_lag12', 'category_4'].agg([np.sum, np.mean, np.max]),
                        how='left',
                        on='card_id')
df_test=df_test.merge(df_nmerchant.groupby('card_id')
                        ['merchant_category_id_y'
                         , 'numerical_1',  'numerical_2', 'avg_sales_lag3', 'avg_purchases_lag3'
                         , 'active_months_lag3', 'avg_sales_lag6', 'avg_purchases_lag6', 'active_months_lag6'
                         , 'avg_sales_lag12', 'avg_purchases_lag12', 'active_months_lag12', 'category_4'].agg([np.sum, np.mean, np.max]),
                      'left', on='card_id')
df_test.shape, df_train.shape

In [ ]:
df_train.to_csv('train.csv', index=False)
df_test.to_csv('test.csv', index=False)

In [ ]:
! ls